In [13]:

import pandas as pd
import numpy as np
import scipy.stats as stats

# Defining the investment stages and their outcome distributions
STAGES = ['Pre-seed', 'Seed', 'Post-seed', 'Series A']
NEW_FOLLOW_ON = ['New', 'Follow-on']
OUTCOME_DISTRIBUTIONS = {
    '0x': {'prob': 0.40, 'value': 0},
    '0x-1x': {'prob': 0.10, 'dist': stats.uniform(loc=0, scale=1)},
    '1x-2x': {'prob': 0.35, 'dist': stats.uniform(loc=1, scale=1)},
    '2x-5x': {'prob': 0.10, 'dist': stats.uniform(loc=2, scale=3)},
    '5x+': {'prob': 0.05, 'dist': stats.truncnorm(a=(5 - 2.5) / 1.5, b=np.inf, loc=2.5, scale=1.5)},
}

# Global variables
CAPITAL = 70*(10**6)  # starting capital
MANAGEMENT_FEE = 0.02  # 2%
YEARS = 10

class VentureCapital:
    def __init__(self, initial_capital, management_fee):
        self.committed_capital = initial_capital  # Total committed capital
        self.capital = initial_capital  # Capital currently on hand
        self.management_fee = management_fee
        self.investments = []  # investments that have not yet matured
        self.yearly_investments = []  # for tracking
        self.yearly_returns = []  # for tracking
        self.distributions_to_lps = 0  # total distributions to LPs
        self.distributions_to_gps = 0  # total distributions to GPs
        self.carry = 0.2  # carried interest
        self.preferred_return = 1.08  # preferred return rate as MoM ratio

    def invest(self, year, stage_allocation, new_follow_on_allocation):
        for stage in STAGES:
            for inv_type in NEW_FOLLOW_ON:
                allocation = stage_allocation.loc[year, stage] * new_follow_on_allocation.loc[year, inv_type]
                investment_amount = self.capital * allocation / 4  # investments are distributed quarterly
                self.investments.append((year + np.random.randint(4, 7), investment_amount, stage, inv_type))  # outcomes are observed after 4 to 6 years
                self.yearly_investments.append((year, stage, inv_type, investment_amount))
                self.capital -= investment_amount

    def observe_outcomes(self, year):
        for i in reversed(range(len(self.investments))):
            outcome_year, investment_amount, stage, inv_type = self.investments[i]
            if year >= outcome_year:
                r = np.random.rand()
                cum_prob = 0
                for outcome_range, outcome_info in OUTCOME_DISTRIBUTIONS.items():
                    cum_prob += outcome_info.get('prob', 0)
                    if r < cum_prob:
                        if 'value' in outcome_info:
                            return_ = outcome_info['value'] * investment_amount
                        else:
                            return_ = outcome_info['dist'].rvs() * investment_amount
                        break

                self.yearly_returns.append((year, investment_amount, return_))
                del self.investments[i]


    def manage_funds(self, year):
        if year > 0:  # management fees don't apply in the first year
            self.capital -= self.capital * self.management_fee

        if self.yearly_returns:
            principal_returned = sum([min(return_, investment) for year_, investment, return_ in self.yearly_returns if year_ <= year])
            profits = sum([max(0, return_ - investment) for year_, investment, return_ in self.yearly_returns if year_ <= year])

            # Return of the principal goes to the LPs first
            self.distributions_to_lps += principal_returned
            self.capital -= principal_returned

            # Then, the preferred return is distributed to the LPs from the profits
            preferred_return_amount = min(profits, self.committed_capital * (self.preferred_return - 1))
            self.distributions_to_lps += preferred_return_amount
            profits -= preferred_return_amount

            # If there's any profit left, the catch-up mechanism is applied
            if profits > 0:
                # Amount that needs to go to the GP for the GP to have received 20% of all profits
                required_catch_up = self.distributions_to_lps * self.carry / (1 - self.carry) - self.distributions_to_gps
                catch_up_amount = min(profits, required_catch_up)
                self.distributions_to_gps += catch_up_amount
                profits -= catch_up_amount

            # If there's still any profit left, the rest of the profits are split according to the carried interest
            if profits > 0:
                self.distributions_to_lps += profits * (1 - self.carry)
                self.distributions_to_gps += profits * self.carry


    def calculate_mom_ratio(self):
        fund_mom_ratio = self.capital / self.committed_capital
        lps_mom_ratio = self.distributions_to_lps / self.committed_capital
        return fund_mom_ratio, lps_mom_ratio

def generate_strategy(years):
    stage_allocations = np.random.dirichlet(np.ones(len(STAGES)), size=years)
    stage_allocations = pd.DataFrame(stage_allocations, columns=STAGES)

    new_follow_on_allocations = np.random.dirichlet(np.ones(len(NEW_FOLLOW_ON)), size=years)
    new_follow_on_allocations = pd.DataFrame(new_follow_on_allocations, columns=NEW_FOLLOW_ON)

    return stage_allocations, new_follow_on_allocations

def simulate_vc(stage_strategy, new_follow_on_strategy):
    vc = VentureCapital(CAPITAL, MANAGEMENT_FEE)
    for year in range(YEARS):
        vc.invest(year, stage_strategy, new_follow_on_strategy)
        vc.observe_outcomes(year)
        vc.manage_funds(year)

    investments_df = pd.DataFrame(vc.yearly_investments, columns=['Year', 'Stage', 'Investment Type', 'Investment'])
    returns_df = pd.DataFrame(vc.yearly_returns, columns=['Year', 'Investment Amount', 'Return'])

    fund_mom_ratio, lps_mom_ratio = vc.calculate_mom_ratio()

    return investments_df, returns_df, fund_mom_ratio, lps_mom_ratio, vc.distributions_to_lps

stage_strategy, new_follow_on_strategy = generate_strategy(YEARS)
investments_df, returns_df, fund_mom_ratio, lps_mom_ratio, distributions_to_lps = simulate_vc(stage_strategy, new_follow_on_strategy)

print(investments_df)
print(returns_df)
print(f'Fund MoM ratio: {fund_mom_ratio}')
print(f'LPs MoM ratio: {lps_mom_ratio}')
print(f'Distributions to LPs: {distributions_to_lps}')





       



    Year      Stage Investment Type    Investment
0      0   Pre-seed             New  2.160402e+05
1      0   Pre-seed       Follow-on  7.204755e+05
2      0       Seed             New  5.608824e+04
3      0       Seed       Follow-on  1.874762e+05
4      0  Post-seed             New  3.087694e+05
..   ...        ...             ...           ...
75     9       Seed       Follow-on -1.280808e+05
76     9  Post-seed             New -1.161042e+05
77     9  Post-seed       Follow-on -9.307716e+03
78     9   Series A             New -5.963035e+06
79     9   Series A       Follow-on -4.093992e+05

[80 rows x 4 columns]
    Year  Investment Amount        Return
0      4       3.087694e+05  5.259802e+05
1      5       1.149867e+06  1.837087e+06
2      5       1.976952e+06  2.013901e+06
3      5       3.993764e+05  3.912060e+05
4      5       1.054993e+07  0.000000e+00
5      5       3.316729e+06  5.194332e+06
6      5       1.874762e+05  6.205945e+05
7      5       5.608824e+04  0.000000e+00